# Task 4 
Repeat TASK3, but now for the scenario of the ray being refracted on a path between two media with differing wave speeds.
Demonstrate that Fermat’s principle of minimizing travel time implies Snell’s law of refraction.
- https://www.youtube.com/watch?v=NIHUbgs4Q_k&t=65s # useful video


In [ ]:
# note: draw out what I mean as well with this code
# import modules
import matplotlib.pyplot as plt
import numpy as np
from math import sqrt

# define constants
n1 = 1.00
n2 = 1.33
y1 = 5
y2 = 30
x2 = 100 # the maximum that x can be
c = 300000000
v1 = c/n1
v2 = c/n2

x = None # the varying constant as of now
t = None

# define t and x
x = np.linspace(0, x2, 1000)
t = [((n1/c) * ((np.sqrt((x**2) + (y1 **2)))) + ((n2/c) * (np.sqrt(((x2 - x)**2) + (y2**2))))) for x in x]# derived through the derivation of this
# plot t against x 
plt.plot(x, t)
plt.show()
# find when dt/dx = 0

# redo the calculations for this (this is annoying me)


# plug that x value in the initial equation

# see whether it is the value of x such that n1 * sin(i) = n2 * sin(r)

# write the list of results in the terminal


## Extension Opportunities
Using the slider from the previous task, I could use this to demonstrate the "Brachistochrone curve" (the curve of fastest descent) be demonstrated by having blocks of increasing refractive indices. 
Somehow extract this curve (or prove that this is the correct curve (how? this seems very difficult to prove (but easy to demonstrate)))